## Преобразование данных

In [ ]:
import numpy as np
import gc
import os
from mne_connectivity import spectral_connectivity_epochs

In [ ]:
input_path = "../data/raw_eeg/"
output_path = "../data_npy"

eeg_fnames = [fname for fname in os.listdir(input_path) if fname.endswith('.fif')]
eeg_fnames = np.array(sorted(eeg_fnames, key=lambda fname: fname.split('_')[0].split('.')[0]))

In [ ]:
# Функция для рассчета коннектом

def comp_eeg_coh_epoch_connectomes(
    raw: mne.io.Raw,
    sfreq: int,
    num_channels: int,
    epoch_duration: int = 15
):
    raw.filter(0.5, 40, fir_design='firwin')
    epochs = mne.make_fixed_length_epochs(raw, duration=epoch_duration, preload=True)
    del raw

    num_epochs = len(epochs)
    freq_bands = {
        'delta': (0.5, 4),
        'theta': (4, 8), 
        'alpha': (8, 12),
        'beta': (12, 30),
        'gamma': (30, 40)
    }

    connectome_size = num_channels * num_channels
    all_connectomes = dict()
    for rhythm, limits in freq_bands.items():
        cur_rhythm_connectomes = np.zeros((num_epochs, connectome_size))
        for epoch_idx, epoch in enumerate(epochs):
            cur_rhythm_connectomes[epoch_idx] = spectral_connectivity_epochs(
                [epoch],
                sfreq=sfreq, 
                method='coh', 
                mode='multitaper', 
                fmin=limits[0], 
                fmax=limits[1], 
                faverage=True, 
                verbose=False
            ).get_data().ravel()
        all_connectomes[rhythm] = cur_rhythm_connectomes
    del epochs
    return all_connectomes

In [ ]:
#  Функция преобразование коннектомы в поток

def matrix_to_signal_flow(A: np.array):
    assert A.shape[0] == A.shape[1]
    
    n = A.shape[0]
    f = np.array([A[i, j] for j in range(n) for i in range(j + 1, n)])[:, np.newaxis]
    
    return f

In [ ]:
# EEG -> flows

flows_list = []
error_files = []
for filename in eeg_fnames:
    my_raw = mne.io.read_raw_fif(input_path + filename)
    my_raw.load_data()
    
    sfreq = my_raw.info['sfreq']
    nchan = my_raw.info['nchan']
    
    try:
        data = comp_eeg_coh_epoch_connectomes(my_raw, sfreq, nchan)
    except Exception as e:
        error_files.append(filename)
        continue
        
    
    del my_raw
    gc.collect()
    print(filename)
    
    for _, f in data.items():
        f = f.reshape((nchan, nchan))
        flow = matrix_to_signal_flow(f)
        flows_list.append(flow)
    
flows = np.concatenate(flows_list, axis=1)

In [ ]:
flows = np.concatenate(flows_list, axis=1)
print(flows.shape)

np.save("flows.npy", flows)